<a href="https://colab.research.google.com/github/DS-Aditya-928/ResNet50_FineTuning/blob/main/ResNet50_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading data

In [8]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

--2024-12-01 13:27:38--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.63.207, 142.250.31.207, 142.251.111.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.63.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   238MB/s    in 0.3s    

2024-12-01 13:27:39 (238 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Reading the Data into arrays

In [21]:
from PIL import Image
import numpy as np

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="training",
  seed=3824,
  image_size=(150, 150),
  )

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="validation",
  seed=3824,
  image_size=(150, 150),
  )

Found 2000 files belonging to 2 classes.
Using 1600 files for training.
Found 2000 files belonging to 2 classes.
Using 400 files for validation.


## Beginning to define the model (this is where you come in, I loaded the pretrained model for you)

In [12]:
import tensorflow as tf
from keras import layers
from keras.layers import Dense, Flatten
from keras import Sequential
from keras.optimizers import Adam

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(150, 150, 3),
    pooling='avg',
    classes=2,
    weights='imagenet'
)

for layer in pretrained_model.layers:
    layer.trainable = False

newModel = Sequential()

newModel.add(pretrained_model)
newModel.add(Dense(2, activation='softmax'))
newModel.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 2048)                │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2)                   │           4,098 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,591,810 (90.00 MB)

 Trainable params: 4,098 (16.01 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [19]:
newModel.compile(optimizer=Adam(learning_rate=0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [20]:
epochs=10
history = newModel.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 164s 3s/step - accuracy: 0.7682 - loss: 0.4973 - val_accuracy: 0.9175 - val_loss: 0.1763
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 152s 3s/step - accuracy: 0.9618 - loss: 0.0986 - val_accuracy: 0.9450 - val_loss: 0.1370
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 214s 3s/step - accuracy: 0.9732 - loss: 0.0651 - val_accuracy: 0.9475 - val_loss: 0.1331
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 189s 3s/step - accuracy: 0.9732 - loss: 0.0687 - val_accuracy: 0.9600 - val_loss: 0.1251
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 151s 3s/step - accuracy: 0.9832 - loss: 0.0438 - val_accuracy: 0.9525 - val_loss: 0.1215
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 151s 3s/step - accuracy: 0.9854 - loss: 0.0393 - val_accuracy: 0.9575 - val_loss: 0.1247
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 202s 3s/step - accuracy: 0.9948 - loss: 0.0265 - val_accuracy: 0.9675 - val_loss: 0.1111
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 162s 3s/step - accuracy: 0.9947 - loss: 0.0242 - val_accuracy: 0.9650 - v